In [ ]:
# Importando Bibliotecas
import pandas as pd
from scipy.stats import zscore
from AnalysisUtils import *
from sklearn.preprocessing import StandardScaler
from scipy.stats import spearmanr
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.impute import KNNImputer

In [ ]:
def encode_string_columns(df):
  # Select columns of type 'object' or 'category'
  non_numeric_columns = df.select_dtypes(include=['object', 'category'])

  for col in non_numeric_columns.columns:
    # Get the unique values in the column
    unique_values = df[col].dropna().unique()
    
    if col == 'come_entre_refeicoes':
      df['dummy_come_entre_refeicoes'] = df[col].str.lower().map({'nao':0, 'as vezes':1, 'frequentemente':2, 'sempre': 3})
      continue
      
    if col == 'consumo_alcool':
      df['dummy_come_entre_refeicoes'] = df[col].str.lower().map({'nao':0, 'as vezes':1, 'frequentemente':2})
      continue

    if df[col].str.lower().isin(['sim', 'nao']).all():
      df[f"dummy_{col}"] = df[col].str.lower().map({'sim': 1, 'nao': 0})
      continue
      
    if len(unique_values) == 2:
      label_encoder = LabelEncoder()
      df[f"dummy_{col}"] = label_encoder.fit_transform(df[col])
      continue
    else:
      df_dummies = pd.get_dummies(df[col], prefix=f"dummy_{col}")
      df = pd.concat([df, df_dummies], axis=1)
      continue
        
  return df

def fill_missing_data(df, method='simple', n_neighbors=5):
  if method == 'simple':
    for col in df.columns:
      if df[col].dtype in ['float64', 'int64']:
        df[col] = df[col].fillna(df[col].mean())
      else:
        if df[col].notna().any():
          df[col] = df[col].fillna(df[col].mode()[0])

  elif method == 'knn':
    # Separate numeric and non-numeric columns
    df_numeric = df.select_dtypes(include=['float64', 'int64'])
    df_non_numeric = df.select_dtypes(exclude=['float64', 'int64'])

    # Apply get_dummies for non-numeric columns to perform one-hot encoding
    df_non_numeric_dummies = pd.get_dummies(df_non_numeric, drop_first=False)

    # Impute using KNN
    imputer_numeric = KNNImputer(n_neighbors=n_neighbors)
    imputer_non_numeric = KNNImputer(n_neighbors=1)
    
    df_imputed_numeric = pd.DataFrame(
      imputer_numeric.fit_transform(df_numeric),
      columns=df_numeric.columns,
      index=df_numeric.index
    )

    df_imputed_non_numeric = pd.DataFrame(
      imputer_non_numeric.fit_transform(df_non_numeric_dummies),
      columns=df_non_numeric_dummies.columns,
      index=df_non_numeric_dummies.index
    )
    # Reverse the one-hot encoding by getting the most frequent category for each column
    df_non_numeric_imputed = pd.DataFrame(index=df_imputed_non_numeric.index)
    for col in df_non_numeric.columns:
      vals_col = df_non_numeric[col].dropna().unique()
      
      dummies = [col + '_' + str(val) for val in vals_col]
      dummies_values = df_imputed_non_numeric[dummies].idxmax(axis=1).apply(lambda x: x.split('_')[-1])
      df_non_numeric_imputed[col] = dummies_values

    # Combine numeric and non-numeric back to the original DataFrame
    df = pd.concat([df_imputed_numeric, df_non_numeric_imputed], axis=1)

  else:
    raise ValueError("Invalid method. Choose from 'simple' or 'knn'.")

  return df

def detect_outliers(df):
  z_scores = np.abs(zscore(df.select_dtypes(include=[np.number])))
  outliers = z_scores > 3  # Consider values with Z-score greater than 3 as outliers
  return outliers.sum()

def analyze_numerical_column(row):
  print_simple_metrics(row)
  boxplot_with_quartiles(row, yscale='linear')
  create_filtered_histograms(row, log=False, filters=None, color='blue', bins=100)

def calculate_group_metric(df, category_columns, numerical_columns, metric='median'):
  result_list = []
  
  for i in range(len(category_columns)):
    for value in df[category_columns[i]].unique():
      group_df = df[df[category_columns[i]] == value]
      
      if metric == 'median':
        group_metric = group_df[numerical_columns].median()
      else:
        group_metric = group_df[numerical_columns].mean()
      
      group_metric['category'] = f"{category_columns[i]}: {value}"
      result_list.append(group_metric)

  result_df = pd.DataFrame(result_list)
  
  return result_df

def plot_sorted_group_metric(result_df, numerical_columns, metric='median'):
  for col in numerical_columns:
    # Sort the result DataFrame by the median of the current column
    sorted_df = result_df.set_index('category').sort_values(by=col).reset_index()

    # Normalize the median values for coloring
    norm = plt.Normalize(sorted_df[col].min(), sorted_df[col].max())
    cmap = plt.cm.viridis  # You can change this to another colormap, e.g., 'plasma', 'inferno', etc.

    # Create a new figure for each numerical column
    plt.figure(figsize=(10, 6))

    # Create the gradient bars by mapping the median values to the color map
    bars = plt.bar(sorted_df['category'], sorted_df[col], color=cmap(norm(sorted_df[col])), width=0.6)

    # Set labels and title for the plot
    plt.xlabel('Category')
    plt.ylabel(f'{metric} of {col}')
    plt.title(f'Sorted {metric}s of {col} by Category')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    # Add a color bar to indicate the gradient scale
    cbar = plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), ax=plt.gca())
    cbar.set_label(f'Median Value of {col}')

    # Show the plot
    plt.show()

def plot_all_category_columns(df, category_columns, color='skyblue', top=10):
  num_columns = len(category_columns)
  rows = (num_columns // 3) + (num_columns % 3 > 0)

  fig, axes = plt.subplots(rows, 3, figsize=(15, 5 * rows))
  axes = axes.flatten()

  for idx, column in enumerate(category_columns):
    ax = axes[idx]

    # Get the value counts for the encoded column
    result_df = df[column].value_counts().reset_index()
    result_df.columns = [column, 'Count']
    result_df['Percentage'] = (result_df['Count'] / result_df['Count'].sum()) * 100

    # Keep only top N values if specified
    result_df = result_df.head(top)

    # Plot the bar chart
    result_df.plot(kind='bar', x=column, y='Count', color=color, ax=ax, legend=False)

    ax.set_title(f"Value Counts of {column}", fontsize=12, pad=20)
    ax.set_xlabel('Values', fontsize=10)
    ax.set_ylabel('Count', fontsize=10)
    ax.tick_params(axis='x', rotation=45)

    for i, (count, pct) in enumerate(zip(result_df['Count'], result_df['Percentage'])):
      ax.text(i, count + 0.5, f'{count} / {pct:.1f}%', ha='center', va='bottom')

  # Remove any empty axes
  for i in range(idx + 1, len(axes)):
    fig.delaxes(axes[i])

  plt.subplots_adjust(hspace=0.5, wspace=0.3)
  plt.tight_layout()
  plt.show()

def scale_numerical_data(df, columns):
  scaler = StandardScaler()
  df[columns] = scaler.fit_transform(df[columns])
  return df

def scale_and_prepare_data(df, categorical_columns, numerical_columns):
  # One-hot encode categorical columns
  df_dummies = pd.get_dummies(df[categorical_columns], drop_first=True)

  # Initialize StandardScaler
  scaler = StandardScaler()

  # Scale the specified numerical columns
  df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

  # Scale the one-hot encoded categorical columns (df_dummies)
  df_dummies_scaled = scaler.fit_transform(df_dummies)

  # Convert the scaled dummies back to a DataFrame and assign correct column names manually
  df_dummies_scaled = pd.DataFrame(df_dummies_scaled, columns=df_dummies.columns, index=df.index)

  # Concatenate the scaled numerical columns with the scaled one-hot encoded columns
  df = pd.concat([df[numerical_columns], df_dummies_scaled], axis=1)

  return df

def add_health_columns(df):
    # Calcular IMC
    df['IMC'] = df['peso'] / (df['altura'] ** 2)

    # Classificar IMG
    def classify_imc(img):
        if img <= 18.5:
            return 'Baixo'
        elif img <= 24.9:
            return 'Normal'
        elif img <= 29.9:
            return 'Sobrepeso'
        else:
            return 'Obesidade'

    df['Class_IMC'] = df['IMC'].apply(classify_imc)

    return df

def plot_and_return_correlation(df, category_columns_dummies, numeric_columns, method='spearman', plot=True):

  df_numeric = df[numeric_columns + category_columns_dummies]

  correlation_matrix = df_numeric.corr(method=method)

  # Plot the heatmap
  if plot:
    plt.figure(figsize=(8, 6))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title(f'Matriz de Correlação ({method.capitalize()})')
    plt.show()

  # Flatten the correlation matrix and sort by absolute value
  corr_pairs = correlation_matrix.unstack()
  sorted_corr = corr_pairs.sort_values(key=lambda x: x.abs(), ascending=False)

  # Exclude self-correlations
  filtered_corr = sorted_corr[sorted_corr.index.get_level_values(0) != sorted_corr.index.get_level_values(1)]

  # Remove duplicates like [A, B] and [B, A]
  unique_pairs = filtered_corr.reset_index()
  unique_pairs['sorted_index'] = unique_pairs.apply(lambda row: tuple(sorted([row['level_0'], row['level_1']])), axis=1)
  unique_pairs = unique_pairs.drop_duplicates(subset='sorted_index').drop(columns='sorted_index')

  # Rename columns for clarity
  unique_pairs.columns = ['Variable 1', 'Variable 2', 'Correlation']

  return unique_pairs, correlation_matrix

def show_changed_missing_values(df1, df2):

  df1_ordered = df1[df2.columns]
  changed = df1_ordered != df2
  changed_rows_df1 = df1_ordered[changed].dropna(how='all')
  changed_rows_df2 = df2[changed].dropna(how='all')

  merged_values = changed_rows_df1.combine(
    changed_rows_df2,
    lambda x1, x2: pd.Series([f"{v1} | {v2}" if pd.notna(v1) and pd.notna(v2) else None for v1, v2 in zip(x1, x2)])
  )

  return merged_values

In [ ]:
numerical_columns = ['idade', 'idade_int', 'altura', 'peso', 'consome_vegetais', 'consome_vegetais_int', 'n_refeicoes_int', 'consumo_diario_agua', 'consumo_diario_agua_int', 'frequencia_atividade_fisica', 'frequencia_atividade_fisica_int', 'tempo_usando_eletronicos', 'tempo_usando_eletronicos_int', 'IMC']

category_columns = ['sexo', 'historico_obesidade_familia', 'consome_comida_calorica', 'come_entre_refeicoes', 'fuma', 'consumo_alcool', 'tipo_transporte', 'Class_IMC', 'tipo_transporte_adaptado']

category_columns_dummies = [
  'dummy_sexo',
  'dummy_historico_obesidade_familia',
  'dummy_consome_comida_calorica',
  'dummy_come_entre_refeicoes',
  'dummy_fuma',
  'dummy_tipo_transporte_adaptado_ativo',
  'dummy_tipo_transporte_adaptado_individual',
  'dummy_tipo_transporte_adaptado_transporte publico'
]

# Comparação de estratégias de inputting

In [ ]:
df_test = pd.read_csv('data/trabalho2_dados_4.csv')
missing_rows_idx = df_test[df_test.isna().any(axis=1)].index.tolist()

df_test_knn = fill_missing_data(df_test, method='knn')
df_test_simple = fill_missing_data(df_test, method='simple')

show_changed_missing_values(df_test_knn, df_test_simple)

## buscando duplicada nova gerada com inputting (gerou para ambos)
ele tinha transporte publico como NaN, mas ao substiruí-lo virou uma duplicada perfeita.

In [ ]:
# analisando duplicata nova gerada pelo fill_missing_data
df_test = pd.read_csv('data/trabalho2_dados_4.csv')

print('Antes do Input')
new_duplicate = df_test.loc[602].copy()
display(df_test[df_test.eq(df_test.loc[1]).all(axis=1)])

df_test = fill_missing_data(df_test) 

print('Depois do Input')
display(df_test.loc[602])
display(df_test[df_test.eq(df_test.loc[1]).all(axis=1)])
new_duplicate

## Pre processando Dados

In [ ]:
# Importando dados
df = pd.read_csv('data/trabalho2_dados_4.csv')
df = fill_missing_data(df, method='knn') # substituindo valores faltantes numéricos pela media e categóricos pela moda


df['idade_int'] = np.floor(df['idade'])
df['n_refeicoes_int'] = df['n_refeicoes'].round()
df['consome_vegetais_int'] = df['consome_vegetais'].round()
df['consumo_diario_agua_int'] = df['consumo_diario_agua'].round()
df['frequencia_atividade_fisica_int'] = df['frequencia_atividade_fisica'].round()
df['tempo_usando_eletronicos_int'] = df['tempo_usando_eletronicos'].round()

df['tipo_transporte_adaptado'] = df['tipo_transporte'].apply(
  lambda x: 'ativo' if x in ['bicicleta', 'andando'] else 'individual' if x in ['carro', 'moto'] else x)

df = add_health_columns(df)
df = encode_string_columns(df) # encodando colunas de string
df = df.drop_duplicates(df, keep='first') # remoção de ~250 entradas duplicadas (substituição de valores faltantes e transformação direta em int nao criaram novas duplicados apenas 1 para o caso de substituição
df

## Detecção de outliers

In [ ]:
detect_outliers(df) 
# Não foram detectados outliers relevantes nas métricas numéricas
# a unica metrica com outliers usando z_score foi a de idade, mas essa distribuição não é normal, e foram apenas 7 encontrados
# nas métricas categóricas, isso removeria certas do categorias do dataset, portanto, não é necessário remover outliers neste dataset

## Análises simples

In [ ]:
plot_all_category_columns(df, category_columns, color='skyblue', top=10)

In [ ]:
for row in numerical_columns:
  analyze_numerical_column(df[row]) 

## Gráficos categorias 

In [ ]:
result_df = calculate_group_metric(df, category_columns, numerical_columns, metric='mean')
plot_sorted_group_metric(result_df, numerical_columns, metric='mean')

## Análise de correlação com spearmann

In [ ]:
sorted_corr, corr_matrix = plot_and_return_correlation(df, category_columns_dummies=category_columns_dummies, numeric_columns=['idade_int', 'altura', 'peso', 'consome_vegetais_int', 'n_refeicoes_int', 'consumo_diario_agua_int', 'frequencia_atividade_fisica_int', 'tempo_usando_eletronicos_int', 'IMC'], method='spearman')
sorted_corr

# analise de influencia de colunas int para correlação

In [ ]:
sorted_corr, corr_matrix = plot_and_return_correlation(df, category_columns_dummies=category_columns_dummies,
                            numeric_columns=['idade_int', 'consome_vegetais_int', 'n_refeicoes_int',
                                             'consumo_diario_agua_int', 'frequencia_atividade_fisica_int',
                                             'tempo_usando_eletronicos_int'], method='spearman', plot=False)

sorted_corr2, corr_matrix2 = plot_and_return_correlation(df, category_columns_dummies=category_columns_dummies,
                                                       numeric_columns=['idade', 'consome_vegetais', 'n_refeicoes', 'consumo_diario_agua','frequencia_atividade_fisica', 'tempo_usando_eletronicos'],
                                                       method='spearman', plot=False)


diff = abs(corr_matrix.to_numpy()) - abs(corr_matrix2.to_numpy())
mask1 = (abs(corr_matrix.to_numpy()) >= 0.3) & (diff > 0)
plt.figure(figsize=(8, 6))
sns.heatmap(mask, annot=True, cmap='coolwarm', fmt=".2f")
plt.title(f'int vs sem int  (diff>0)')
plt.show()


mask2 = (abs(corr_matrix.to_numpy()) >= 0.3) & (diff < 0)
plt.figure(figsize=(8, 6))
sns.heatmap(mask2, annot=True, cmap='coolwarm', fmt=".2f")
plt.title(f'int vs sem int (diff<0)')
plt.show()
